# PROBLEM 1: Confirmation of competition contents

1. What to learn and what to predict?
        We use Homecredit's data set to learn and predict their clients' repayment abilities.

2. What kind of file to create and submit to Kaggle?
        We summit a file that show our predicted probability of customer being type 1 (slow on dept payment) for all test indexes. For example, SK_ID_CURR = 100001, TARGET = 0.1

3. What kind of index value will be used to evaluate the submissions?
        Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

# PROBLEM 2: Learning and verification

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_csv('application_train.csv')
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
print('Data shape: {}'.format(data.shape))

Data shape: (307511, 122)


NULL CHECK

In [4]:
rows = data.shape[0]
null_counts = data.isnull().sum()
null_columns = data.columns[null_counts > 0]
print('Number of columns with null value:', len(null_columns))
null_columns

Number of columns with null value: 67


Index(['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'OWN_CAR_AGE',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG',
       'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG',
       'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG',
       'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE',
       'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
       'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE',
       'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE',
       'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
       'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI',
       'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMI

Finding unusable columns

In [5]:
null_counts_normalized = data[null_columns].isnull().sum() / rows * 100
unusable_columns = null_counts_normalized.index[null_counts_normalized > 50]
print('Number of unusable columns: ', len(unusable_columns))
unusable_columns

Number of unusable columns:  41


Index(['OWN_CAR_AGE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG',
       'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
       'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG',
       'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMIN_MODE',
       'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
       'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI',
       'BASEMENTAREA_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI',
       'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI',
       'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI',
       'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE',
       'WALLSMATERIAL_MODE'],
      dtype='object')

After Dropping Unsusable Columns

In [6]:
data_dropped = data.drop(columns = unusable_columns)
print('Number of columns after dropping:', data_dropped.shape[1])
data_dropped.columns

Number of columns after dropping: 81


Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL',
       'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
       'ORGANIZATION_TYPE', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
       'YEARS_BEGINEXPLUATATION_AVG', 'FLOOR

Checking Attribute types and fill up null

In [7]:
fill_values = data_dropped.mode().iloc[0, :]
data_filled = data_dropped.fillna(fill_values)
print("If there is any null value left:", data_filled.isnull().sum().any())    

If there is any null value left: False


Encoding label of columns that are object type

In [8]:
object_cols = data_filled.columns[data_filled.dtypes == 'object']
data_filled[object_cols].nunique()

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
EMERGENCYSTATE_MODE            2
dtype: int64

In [9]:
from sklearn.preprocessing import LabelEncoder
for col in object_cols:
    data_filled[col] = LabelEncoder().fit_transform(data_filled[col])
data_filled

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,1,0,1,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,0,0,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,1,1,1,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,0,0,1,0,135000.0,312682.5,29686.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,0,1,0,1,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,1,0,0,0,157500.0,254700.0,27558.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
307507,456252,0,0,0,0,1,0,72000.0,269550.0,12001.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
307508,456253,0,0,0,0,1,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,0,0,0,1,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


I will pick 30 attributes that have highest corralation.

In [10]:
data_corr = data_filled.corr()
picked_cols = data_corr['TARGET'].abs().sort_values(ascending = False).head(31)[1:31].index
picked_cols

Index(['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH',
       'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT',
       'DAYS_LAST_PHONE_CHANGE', 'NAME_EDUCATION_TYPE', 'CODE_GENDER',
       'DAYS_ID_PUBLISH', 'REG_CITY_NOT_WORK_CITY', 'NAME_INCOME_TYPE',
       'FLAG_EMP_PHONE', 'DAYS_EMPLOYED', 'REG_CITY_NOT_LIVE_CITY',
       'FLAG_DOCUMENT_3', 'DAYS_REGISTRATION', 'TOTALAREA_MODE',
       'AMT_GOODS_PRICE', 'FLOORSMAX_AVG', 'FLOORSMAX_MEDI', 'FLOORSMAX_MODE',
       'REGION_POPULATION_RELATIVE', 'NAME_HOUSING_TYPE',
       'LIVE_CITY_NOT_WORK_CITY', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'DEF_60_CNT_SOCIAL_CIRCLE', 'NAME_CONTRACT_TYPE', 'ORGANIZATION_TYPE',
       'AMT_CREDIT', 'FLAG_DOCUMENT_6'],
      dtype='object')

In [11]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

X = data_filled[picked_cols]
y = data_filled['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

print('Shape of X_train, X_test, y_train, y_test: {}; {}, {}, {}'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

cls = SGDClassifier()
cls.fit(X_train,y_train)
y_pred = cls.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))

Shape of X_train, X_test, y_train, y_test: (246008, 30); (61503, 30), (246008,), (61503,)
Accuracy:  0.9188982651252784


Evaluation

In [12]:
from sklearn.calibration import CalibratedClassifierCV
calibrator = CalibratedClassifierCV(cls, cv = 'prefit')
model = calibrator.fit(X_train,y_train)
print('Predict probability:')
model.predict_proba(X_test)

Predict probability:


array([[0.88209173, 0.11790827],
       [0.94463016, 0.05536984],
       [0.96312438, 0.03687562],
       ...,
       [0.87574858, 0.12425142],
       [0.92051054, 0.07948946],
       [0.8929768 , 0.1070232 ]])

# PROBLEM 3: Estimation on test data

In [13]:
data_test = pd.read_csv('application_test.csv')
IDs = data_test['SK_ID_CURR']
rows = data_test.shape[0]
data_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


picking columns, filling null cells and label encoding

In [14]:
data_test = data_test[picked_cols]
fill_values = data_test.mode().iloc[0, :]
data_test = data_test.fillna(fill_values)
print("If there is any null value left:", data_test.isnull().sum().any()) 

object_cols = data_test.columns[data_test.dtypes == 'object']
for col in object_cols:
    data_test[col] = LabelEncoder().fit_transform(data_test[col])
  
data_test

If there is any null value left: False


,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,NAME_EDUCATION_TYPE,CODE_GENDER,DAYS_ID_PUBLISH,REG_CITY_NOT_WORK_CITY,...,FLOORSMAX_MODE,REGION_POPULATION_RELATIVE,NAME_HOUSING_TYPE,LIVE_CITY_NOT_WORK_CITY,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,NAME_CONTRACT_TYPE,ORGANIZATION_TYPE,AMT_CREDIT,FLAG_DOCUMENT_6
0,0.789654,0.159520,-19241,2,2,-1740.0,1,0,-812,0,...,0.1250,0.018850,1,0,0.0,0.0,0,28,568800.0,0
1,0.291656,0.432962,-18064,2,2,0.0,4,1,-1623,0,...,0.1667,0.035792,1,0,0.0,0.0,0,42,222768.0,0
2,0.699787,0.610991,-20038,2,2,-856.0,1,1,-3503,0,...,0.1667,0.019101,1,0,0.0,0.0,0,54,663264.0,0
3,0.509677,0.612704,-13976,2,2,-1805.0,4,0,-4208,0,...,0.3750,0.026392,1,0,0.0,0.0,0,5,1575000.0,0
4,0.425687,0.706205,-13040,2,2,-821.0,4,1,-4262,1,...,0.1667,0.010032,1,1,0.0,0.0,0,5,625500.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0.648575,0.643026,-19970,3,3,-684.0,4,0,-3399,0,...,0.1667,0.002042,1,0,0.0,0.0,0,33,412560.0,0
48740,0.684596,0.706205,-11186,2,2,0.0,4,0,-3003,1,...,0.1667,0.035792,1,1,0.0,0.0,0,51,622413.0,0
48741,0.632770,0.283712,-15922,2,2,-838.0,4,0,-1504,0,...,0.3333,0.026392,1,0,0.0,0.0,0,5,315000.0,0
48742,0.445701,0.595456,-13968,2,2,-2308.0,1,1,-1364,1,...,0.6250,0.018850,1,1,0.0,0.0,0,42,450000.0,0


In [15]:
data_test = StandardScaler().fit_transform(data_test)
result = model.predict_proba(data_test)
result

array([[0.91969199, 0.08030801],
       [0.87323716, 0.12676284],
       [0.92603145, 0.07396855],
       ...,
       [0.92695494, 0.07304506],
       [0.9459884 , 0.0540116 ],
       [0.85320482, 0.14679518]])

In [16]:
submit = pd.DataFrame(data = result[:, 0], index = IDs, columns = ['TARGET'])
submit.to_csv('submit_kaggle.csv')
submit.head()

,TARGET
SK_ID_CURR,
100001,0.919692
100005,0.873237
100013,0.926031
100028,0.954472
100038,0.907758


I submitted the result to Kaggle and got the score of 0.41679

# Problem 4: Feature engineering

1. Considering more carefully when deciding which columns to be dropped. 
<br>
2. Finding more appropriate values to fill in null cells
<br>
3. Picking attributes that actually have meaning to the target not just depend on the correlation.
<br>
4. Using more training method to see which is the best for this problem.